In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle as pkl
import matplotlib.pyplot as plt
import lightgbm as lgb
import sklearn.model_selection as GridSearchCV
import sklearn.model_selection as train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/PV,UV流量预测竞赛数据/kpi_train.csv
/kaggle/input/PV,UV流量预测竞赛数据/event_detail.csv
/kaggle/input/PV,UV流量预测竞赛数据/user.csv


In [2]:
kpi = pd.read_csv('/kaggle/input/PV,UV流量预测竞赛数据/kpi_train.csv')
event = pd.read_csv('/kaggle/input/PV,UV流量预测竞赛数据/event_detail.csv')
user = pd.read_csv('/kaggle/input/PV,UV流量预测竞赛数据/user.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (16,17,18,28,32,34,38,41,42,45,49,50,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()  ## 最大最小范围
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':  ## int型数据处理
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:  ## float型数据处理
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        #else:
            #df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB --> {:.2f} MB (Decreased by {:.1f}%)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
kpi = reduce_mem_usage(kpi)
event = reduce_mem_usage(event)
user = reduce_mem_usage(user)

Memory usage of dataframe is 0.01 MB --> 0.01 MB (Decreased by 49.5%)
Memory usage of dataframe is 1394.44 MB --> 1086.68 MB (Decreased by 22.1%)
Memory usage of dataframe is 134.74 MB --> 110.83 MB (Decreased by 17.7%)


In [5]:
print(kpi.shape,event.shape,user.shape)

(400, 4) (2855816, 64) (569695, 31)


In [6]:
from datetime import datetime

In [7]:
kpi['date'] = kpi['date'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))
# kpi['year_month'] = kpi['date'].dt.year.astype(str)+ '/' + kpi['date'].dt.month.astype(str)
# kpi['week'] = kpi['date'].dt.dayofweek

In [8]:
kpi.head()

,date,event_type,pv,uv
0,2018-11-01,$pageview,6549,674
1,2018-11-01,reg_input_success,29,20
2,2018-11-02,$pageview,3972,581
3,2018-11-02,reg_input_success,34,21
4,2018-11-03,$pageview,1446,395


In [9]:
page = kpi[kpi['event_type']=='$pageview']
reg = kpi[kpi['event_type']=='reg_input_success']

In [10]:
page.loc[:,'rpv'] = reg['pv'].values
page.loc[:,'ruv'] = reg['uv'].values
page.rename(columns={'pv':'ppv','uv':'puv'},inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/panda

In [11]:
page.head()

,date,event_type,ppv,puv,rpv,ruv
0,2018-11-01,$pageview,6549,674,29,20
2,2018-11-02,$pageview,3972,581,34,21
4,2018-11-03,$pageview,1446,395,8,8
6,2018-11-04,$pageview,1387,411,7,7
8,2018-11-05,$pageview,3656,800,14,14


In [12]:
data = page.drop(columns='event_type')

In [13]:
data.head()

,date,ppv,puv,rpv,ruv
0,2018-11-01,6549,674,29,20
2,2018-11-02,3972,581,34,21
4,2018-11-03,1446,395,8,8
6,2018-11-04,1387,411,7,7
8,2018-11-05,3656,800,14,14


In [14]:
data['month'] = data['date'].dt.year.astype(str)+'/'+data['date'].dt.month.astype(str)
data['week'] = data['date'].dt.dayofweek

In [15]:
# data['prate'] = data['puv']/data['ppv']
# data['rrate'] = data['ruv']/data['rpv']

In [16]:
def Shift(df,typ):
    for t in typ:
        for i in range(1,8):
            df[t+"_"+str(i)] = df[t].shift(i)
    return df
data = Shift(data,['ppv','puv','rpv','ruv'])

In [17]:
data.fillna(0)

,date,ppv,puv,rpv,ruv,month,week,ppv_1,ppv_2,ppv_3,...,rpv_5,rpv_6,rpv_7,ruv_1,ruv_2,ruv_3,ruv_4,ruv_5,ruv_6,ruv_7
0,2018-11-01,6549,674,29,20,2018/11,3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-11-02,3972,581,34,21,2018/11,4,6549.0,0.0,0.0,...,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-11-03,1446,395,8,8,2018/11,5,3972.0,6549.0,0.0,...,0.0,0.0,0.0,21.0,20.0,0.0,0.0,0.0,0.0,0.0
6,2018-11-04,1387,411,7,7,2018/11,6,1446.0,3972.0,6549.0,...,0.0,0.0,0.0,8.0,21.0,20.0,0.0,0.0,0.0,0.0
8,2018-11-05,3656,800,14,14,2018/11,0,1387.0,1446.0,3972.0,...,0.0,0.0,0.0,7.0,8.0,21.0,20.0,0.0,0.0,0.0
10,2018-11-06,4257,1012,33,33,2018/11,1,3656.0,1387.0,1446.0,...,29.0,0.0,0.0,14.0,7.0,8.0,21.0,20.0,0.0,0.0
12,2018-11-07,3813,856,25,25,2018/11,2,4257.0,3656.0,1387.0,...,34.0,29.0,0.0,33.0,14.0,7.0,8.0,21.0,20.0,0.0
14,2018-11-08,4176,875,29,29,2018/11,3,3813.0,4257.0,3656.0,...,8.0,34.0,29.0,25.0,33.0,14.0,7.0,8.0,21.0,20.0
16,2018-11-09,3518,650,24,24,2018/11,4,4176.0,3813.0,4257.0,...,7.0,8.0,34.0,29.0,25.0,33.0,14.0,7.0,8.0,21.0
18,2018-11-10,481,215,7,7,2018/11,5,3518.0,4176.0,3813.0,...,14.0,7.0,8.0,24.0,29.0,25.0,33.0,14.0,7.0,8.0


In [18]:
data = data.iloc[7:]

In [19]:
data.shape

(193, 35)

In [20]:
# fig,ax = plt.subplots(4, 1, figsize=(20,12))
# kpi[kpi['event_type']=='$pageview']['pv'].to_frame().plot.bar(ax=ax[0])
# kpi[kpi['event_type']=='$pageview']['uv'].to_frame().plot.bar(ax=ax[1])
# kpi[kpi['event_type']=='reg_input_success']['pv'].to_frame().plot.bar(ax=ax[2])
# kpi[kpi['event_type']=='reg_input_success']['uv'].to_frame().plot.bar(ax=ax[3])

In [21]:
# fig,ax = plt.subplots(4, 1, figsize=(20,12))
# kpi[kpi['event_type']=='$pageview'].groupby('year_month')['pv'].mean().to_frame().plot.bar(ax=ax[0])
# kpi[kpi['event_type']=='$pageview'].groupby('year_month')['uv'].mean().to_frame().plot.bar(ax=ax[1])
# kpi[kpi['event_type']=='reg_input_success'].groupby('year_month')['pv'].mean().to_frame().plot.bar(ax=ax[2])
# kpi[kpi['event_type']=='reg_input_success'].groupby('year_month')['uv'].mean().to_frame().plot.bar(ax=ax[3])
# # 随着时间的增长，各项指标的变化也是有趋势的

In [22]:
# fig,ax = plt.subplots(4, 1, figsize=(20,12))
# kpi[kpi['event_type']=='$pageview'].groupby('week')['pv'].mean().to_frame().plot.bar(ax=ax[0])
# kpi[kpi['event_type']=='$pageview'].groupby('week')['uv'].mean().to_frame().plot.bar(ax=ax[1])
# kpi[kpi['event_type']=='reg_input_success'].groupby('week')['pv'].mean().to_frame().plot.bar(ax=ax[2])
# kpi[kpi['event_type']=='reg_input_success'].groupby('week')['uv'].mean().to_frame().plot.bar(ax=ax[3])
# # 周0到周6的变化是有趋势的

In [23]:
# fig,ax = plt.subplots(4, 1, figsize=(20,12))
# kpi[kpi['event_type']=='$pageview']['pv'].to_frame().plot.line(ax=ax[0])
# kpi[kpi['event_type']=='$pageview']['uv'].to_frame().plot.line(ax=ax[1])
# kpi[kpi['event_type']=='reg_input_success']['pv'].to_frame().plot.line(ax=ax[2])
# kpi[kpi['event_type']=='reg_input_success']['uv'].to_frame().plot.line(ax=ax[3])
# page = kpi[kpi['event_type']=='$pageview']
# reg = kpi[kpi['event_type']=='reg_input_success']

In [24]:
# page_pv = page.drop(columns='uv')
# page_uv = page.drop(columns='pv')

# reg_pv = reg.drop(columns='uv')
# reg_uv = reg.drop(columns='pv')

In [25]:
# def Shift(df,typ):
#     for i in range(1,8):
#         df[str(i)] = df[typ].shift(i)
#     return df
# page_pv = Shift(page_pv,'pv')
# page_uv = Shift(page_uv,'uv')
# reg_pv = Shift(reg_pv,'pv')
# reg_uv = Shift(reg_uv,'uv')

In [26]:
# page_pv

In [27]:
# page_pv = page_pv.iloc[7:]
# page_uv = page_uv.iloc[7:]
# reg_pv = reg_pv.iloc[7:]
# reg_uv = reg_uv.iloc[7:]

In [28]:
# page_pv['week'].astype('category')
# page_uv['week'].astype('category')
# reg_pv['week'].astype('category')
# reg_uv['week'].astype('category')

In [29]:
# def change(df):
#     for i in range(1,8):
#         page_pv[str(i)].astype(np.int)
#     return df
# page_pv = change(page_pv)

In [30]:
data['week'].astype('category')
train_, test_ = data.iloc[:-7], data.iloc[-7:]

In [31]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [32]:
model1 = lgb.LGBMRegressor(objective='regression',max_depth=4,num_leaves=5,learning_rate=0.03,n_estimators=300,
                          min_data_in_leaf=5,lambda_l2=0.7,lambda_l1=0.6,feature_fraction=0.8,bagging_fraction=0.3,bagging_freq=6,n_jobs=-1,)
model1.fit(train_.iloc[:,6:],train_.iloc[:,1])
res = model1.predict(test_.iloc[:,6:])
print('Test Score:',mean_squared_error(test_.iloc[:,1],res))
print('------------------------------------')

params = {
#     'max_depth':np.arange(2,5,1),
#     'num_leaves':np.arange(2,7,1),
#     'n_estimators':np.arange(100,1000,100),
#     'learning_rate':[0.3,0.1,0.05,0.03,0.01,0.003,0.001],
#     'min_data_in_leaf':np.arange(2,10,1),
#     'lambda_l2':np.arange(0,1,.1),
#     'lambda_l1':np.arange(0,1,.1),
#     'feature_fraction':np.arange(0.1,1,.1),
#     'bagging_fraction':np.arange(0.1,1,.1),
#     'bagging_freq':np.arange(1,10,1),
}
cv = GridSearchCV(model1,params,scoring='neg_mean_squared_error',refit=True,cv=3,verbose=1)

cv.fit(train_.iloc[:,6:],train_.iloc[:,1])

print('Best Params:',cv.best_params_)
print('Best Score:',cv.best_score_)

Test Score: 1082032.5811134712
------------------------------------
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Params: {}
Best Score: -5779247.640599722


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s finished


In [33]:
model2 = lgb.LGBMRegressor(objective='regression',max_depth=2,num_leaves=4,learning_rate=0.03,n_estimators=300,
                          min_data_in_leaf=9,lambda_l2=0.0,lambda_l1=0.0,feature_fraction=0.3,bagging_fraction=0.3,bagging_freq=9,n_jobs=-1,)
model2.fit(train_.iloc[:,6:],train_.iloc[:,2])
res = model2.predict(test_.iloc[:,6:])
print('Test Score:',mean_squared_error(test_.iloc[:,2],res))
print('------------------------------------')

params = {
#     'max_depth':np.arange(1,5,1),
#     'num_leaves':np.arange(2,7,1),
#     'n_estimators':np.arange(100,1000,100),
#     'learning_rate':[0.3,0.1,0.05,0.03,0.01,0.003,0.001],
#     'min_data_in_leaf':np.arange(5,10,1),
#     'lambda_l2':np.arange(0,1,.1),
#     'lambda_l1':np.arange(0,1,.1),
#     'feature_fraction':np.arange(0.1,0.5,.1),
#     'bagging_fraction':np.arange(0.1,0.4,.1),
#     'bagging_freq':np.arange(1,20,1),
}
cv = GridSearchCV(model2,params,scoring='neg_mean_squared_error',refit=True,cv=3,verbose=1)

cv.fit(train_.iloc[:,6:],train_.iloc[:,2])

print('Best Params:',cv.best_params_)
print('Best Score:',cv.best_score_)

Test Score: 215618.35228624722
------------------------------------
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Params: {}
Best Score: -455113.6789972899


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s finished


In [34]:
from sklearn.model_selection import RandomizedSearchCV

In [35]:
model3 = lgb.LGBMRegressor(objective='regression',max_depth=4,num_leaves=5,learning_rate=0.03,n_estimators=300,
                          min_data_in_leaf=5,lambda_l2=0.0,lambda_l1=0.7,feature_fraction=0.4,bagging_fraction=0.4,bagging_freq=3,n_jobs=-1,)
model3.fit(train_.iloc[:,6:],train_.iloc[:,3])
res = model3.predict(test_.iloc[:,6:])
print('Test Score:',mean_squared_error(test_.iloc[:,3],res))
print('------------------------------------')

params = {
#     'max_depth':np.arange(1,6,1),
#     'num_leaves':np.arange(2,10,1),
#     'n_estimators':np.arange(100,1000,100),
#     'learning_rate':[0.3,0.1,0.05,0.03,0.01,0.003,0.001],
#     'min_data_in_leaf':np.arange(3,7,1),
#     'lambda_l2':np.arange(0,1,.1),
#     'lambda_l1':np.arange(0,1.1,.1),
#     'feature_fraction':np.arange(0.1,0.5,.1),
#     'bagging_fraction':np.arange(0.1,0.4,.1),
#     'bagging_freq':np.arange(1,10,1),
}
cv = GridSearchCV(model3,params,scoring='neg_mean_squared_error',refit=True,cv=3,verbose=1)

cv.fit(train_.iloc[:,6:],train_.iloc[:,3])

print('Best Params:',cv.best_params_)
print('Best Score:',cv.best_score_)

Test Score: 415.6441621499915
------------------------------------
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Params: {}
Best Score: -646.2342795717343


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s finished


In [36]:
model4 = lgb.LGBMRegressor(objective='regression',max_depth=2,num_leaves=3,learning_rate=0.1,n_estimators=600,
                          min_data_in_leaf=2,lambda_l2=0.2,lambda_l1=0.5,feature_fraction=0.4,bagging_fraction=0.4,bagging_freq=8,n_jobs=-1,)
model4.fit(train_.iloc[:,6:],train_.iloc[:,4])
res = model4.predict(test_.iloc[:,6:])
print('Test Score:',mean_squared_error(test_.iloc[:,4],res))
print('------------------------------------')

params = {
#     'max_depth':np.arange(1,4,1),
#     'num_leaves':np.arange(2,4,1),
#     'n_estimators':np.arange(100,1000,100),
#     'learning_rate':[0.3,0.1,0.05,0.03,0.01,0.003,0.001],
#     'min_data_in_leaf':np.arange(1,5,1),
#     'lambda_l2':np.arange(0,1,.1),
#     'lambda_l1':np.arange(0,1,.1),
#     'feature_fraction':np.arange(0.1,0.5,.1),
#     'bagging_fraction':np.arange(0.1,0.4,.1),
#     'bagging_freq':np.arange(1,10,1),
}
cv = GridSearchCV(model4,params,scoring='neg_mean_squared_error',refit=True,cv=3,verbose=1)

cv.fit(data.iloc[:,6:],data.iloc[:,4])

print('Best Params:',cv.best_params_)
print('Best Score:',cv.best_score_)

Test Score: 424.40004713881086
------------------------------------
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Params: {}
Best Score: -439.9982709567492


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [37]:
ppv = test_['ppv'].values.tolist()
puv = test_['puv'].values.tolist()
rpv = test_['rpv'].values.tolist()
ruv = test_['ruv'].values.tolist()

In [38]:
def Predict(model1,model2,model3,model4,test):
    ppv = test['ppv'].values.tolist()
    puv = test['puv'].values.tolist()
    rpv = test['rpv'].values.tolist()
    ruv = test['ruv'].values.tolist()
    for i in range(7):
        data = ppv[i:]+puv[i:]+rpv[i:]+ruv[i:]
        res1 = model1.predict([[i]+data]).tolist()[0]
        res2 = model2.predict([[i]+data]).tolist()[0]
        res3 = model3.predict([[i]+data]).tolist()[0]
        res4 = model4.predict([[i]+data]).tolist()[0]
        ppv.append(res1)
        puv.append(res2)
        rpv.append(res3)
        ruv.append(res4)
    return ppv[-7:],puv[-7:],rpv[-7:],ruv[-7:]
ppvs,puvs,rpvs,ruvs = Predict(model1,model2,model3,model4,test_)

In [39]:
print(ppvs)
print(puvs)
print(rpvs)
print(ruvs)

[10362.711336357279, 11138.78117525372, 11920.113268821166, 13446.14339953364, 13076.10253182975, 9456.461168599346, 6358.981516256265]
[2917.6422207392134, 3048.0092277809954, 3087.798949943914, 3317.5733759241803, 3063.413178599518, 2641.3261197993784, 2418.9815612404673]
[92.37532684704811, 109.37881097445673, 117.62336871963299, 114.9701342672063, 99.5210471293344, 73.22005770334039, 53.62861970091223]
[77.58246279205738, 69.22553912625813, 112.45579893935616, 104.97471645919717, 93.81868814771057, 89.7532855897135, 64.18159707134491]


In [40]:
train_, test_ = page_uv.iloc[:-7], page_uv.iloc[-7:]

NameError: name 'page_uv' is not defined

In [41]:
model = lgb.LGBMRegressor(objective='regression',max_depth=1,num_leaves=2,learning_rate=0.03,n_estimators=300,
                          min_data_in_leaf=11,lambda_l2=0.2,lambda_l1=0.0,feature_fraction=0.8,bagging_fraction=0.6,bagging_freq=4,n_jobs=-1,)
model.fit(train_.iloc[:,4:],train_.iloc[:,2])
res = model.predict(test_.iloc[:,4:])
print('Test Score:',mean_squared_error(test_.iloc[:,2],res))
print('------------------------------------')

params = {
#     'max_depth':np.arange(1,5,1),
#     'num_leaves':np.arange(2,5,1),
#     'n_estimators':np.arange(100,1000,100),
#     'learning_rate':[0.3,0.1,0.05,0.03,0.01,0.003,0.001],
#     'min_data_in_leaf':np.arange(10,20,1),
    'lambda_l2':np.arange(0,1,.1),
#     'lambda_l1':np.arange(0,1,.1),
#     'feature_fraction':np.arange(0.1,1,.1),
#     'bagging_fraction':np.arange(0.1,1,.1),
#     'bagging_freq':np.arange(1,10,1),
}
cv = RandomizedSearchCV(model,params,scoring='neg_mean_squared_error',n_iter=1000,refit=True,cv=3,verbose=1)

cv.fit(train_.iloc[:,4:],train_.iloc[:,2])

print('Best Params:',cv.best_params_)
print('Best Score:',cv.best_score_)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in fields month

In [42]:
def Predict(model,test,typ):
    data = test_[typ].values.tolist()
    for i in range(7):
        print([i] + data[i:])
        res = model.predict([[i]+data[i:]]).tolist()[0]
        print(res)
        data.append(res)
    return data[-7:]
Predict(model,test_,'uv')

KeyError: 'uv'

In [43]:
train_, test_ = reg_pv.iloc[:-7], reg_pv.iloc[-7:]

NameError: name 'reg_pv' is not defined

In [44]:
model = lgb.LGBMRegressor(objective='regression',max_depth=2,num_leaves=4,learning_rate=0.03,n_estimators=300,
                          min_data_in_leaf=13,lambda_l2=0.5,lambda_l1=0.8,feature_fraction=0.7,bagging_fraction=0.7,bagging_freq=7,n_jobs=-1,)
model.fit(train_.iloc[:,4:],train_.iloc[:,2])
res = model.predict(test_.iloc[:,4:])
print('Test Score:',mean_squared_error(test_.iloc[:,2],res))
print('------------------------------------')

params = {
    'max_depth':np.arange(1,4,1),
    'num_leaves':np.arange(2,5,1),
    'n_estimators':np.arange(200,500,100),
    'learning_rate':[0.05,0.03,0.01],
    'min_data_in_leaf':np.arange(11,15,1),
    'lambda_l2':np.arange(0.3,0.7,.1),
    'lambda_l1':np.arange(0.6,0.9,.1),
    'feature_fraction':np.arange(0.5,0.9,.1),
    'bagging_fraction':np.arange(0.5,0.9,.1),
    'bagging_freq':np.arange(5,9,1),
}
cv = RandomizedSearchCV(model,params,scoring='neg_mean_squared_error',n_iter=100,refit=True,cv=3,verbose=1)

cv.fit(train_.iloc[:,4:],train_.iloc[:,2])

print('Best Params:',cv.best_params_)
print('Best Score:',cv.best_score_)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in fields month

In [45]:
def Predict(model,test,typ):
    data = test_[typ].values.tolist()
    for i in range(7):
        print([i] + data[i:])
        res = model.predict([[i]+data[i:]]).tolist()[0]
        print(res)
        data.append(res)
    return data[-7:]
Predict(model,test_,'pv')

KeyError: 'pv'

In [46]:
train_, test_ = reg_uv.iloc[:-7], reg_uv.iloc[-7:]

NameError: name 'reg_uv' is not defined

In [47]:
model = lgb.LGBMRegressor(objective='regression',max_depth=3,num_leaves=4,learning_rate=0.03,n_estimators=200,
                          min_data_in_leaf=13,lambda_l2=0.1,lambda_l1=0.0,feature_fraction=0.7,bagging_fraction=0.7,bagging_freq=7,n_jobs=-1,)
model.fit(train_.iloc[:,4:],train_.iloc[:,2])
res = model.predict(test_.iloc[:,4:])
print('Test Score:',mean_squared_error(test_.iloc[:,2],res))
print('------------------------------------')

params = {
#     'max_depth':np.arange(1,4,1),
#     'num_leaves':np.arange(2,5,1),
    'n_estimators':np.arange(100,1000,100),
    'learning_rate':[0.3,0.1,0.05,0.03,0.01,0.003,0.001],
#     'min_data_in_leaf':np.arange(10,20,1),
#     'lambda_l2':np.arange(0,1,.1),
#     'lambda_l1':np.arange(0,1,.1),
#     'feature_fraction':np.arange(0.1,1,.1),
#     'bagging_fraction':np.arange(0.1,1,.1),
#     'bagging_freq':np.arange(1,10,1),
}
cv = GridSearchCV(model,params,scoring='neg_mean_squared_error',refit=True,cv=3,verbose=1)

cv.fit(train_.iloc[:,4:],train_.iloc[:,2])

print('Best Params:',cv.best_params_)
print('Best Score:',cv.best_score_)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in fields month

In [48]:
def Predict(model,test,typ):
    data = test_[typ].values.tolist()
    for i in range(7):
        print([i] + data[i:])
        res = model.predict([[i]+data[i:]]).tolist()[0]
        print(res)
        data.append(res)
    return data[-7:]
Predict(model,test_,'uv')

KeyError: 'uv'

In [49]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569695 entries, 0 to 569694
Data columns (total 31 columns):
distinct_id              569644 non-null float32
xwho                     555856 non-null object
xwhen                    555185 non-null float32
$lib                     551478 non-null object
$lib_version             550807 non-null object
$original_id             28213 non-null object
$platform                550787 non-null object
$first_visit_time        549706 non-null float32
$signup_time             18965 non-null float32
datafrom                 13669 non-null object
data_source              707 non-null object
$first_visit_language    497713 non-null object
$browser_version         2864 non-null object
$os_version              2966 non-null object
$web_crawler             3000 non-null float16
$is_first_day            3000 non-null float16
$os                      2972 non-null object
$screen_width            3000 non-null float16
$startup_time            3000 non-nu

In [50]:
res = []
for i in range(7):
    temp = []
    temp.append('201905'+str(20+i))
    temp.append('$pageview')
    temp.append(page[i][0])
    temp.append(page[i][1])
    res.append(temp)
    temp1 = []
    temp1.append('201905'+str(20+i))
    temp1.append('reg_input_success')
    temp1.append(reg[i][0])
    temp1.append(reg[i][1])
    res.append(temp1)
res = np.array(res)
res = pd.DataFrame(res,columns=['date','event_type','pv','uv'],index=None)
res.to_csv('output.csv',index=False)
res

KeyError: 0